In [11]:
import string
import nltk
import pandas as pd
from nltk import ngrams
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
import re
import sys
import emoji
import docx2txt
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\George
[nltk_data]     Sidorov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\George
[nltk_data]     Sidorov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
class Text_n:

    def __init__(self, initial_text=None):

        self.initial_text = initial_text
        self.initial_text_without_emoji = None

        self.stemmed_text = None
        self.tokenized_text = None

        self.list_gramms_stemmed = []
        self.list_gramms_tokens = []

        self.list_unigramm_stemmed = []
        self.list_unigramm_tokens = []

        self.dict_gramms = {}
        self.sorted_dict_gramms = {}

        self.final_dict = {}

        self.df = pd.DataFrame()

    # ---------------------------- Clean initial text from emojis ------------------------------- #

    def emoji_cleaner(self):
        
        allchars = [str for str in self.initial_text]
        emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
        self.initial_text_without_emoji = ' '.join([str for str in self.initial_text.split() if not any(i in str for i in emoji_list)])
            
    # ---------------------------- Stemming initial text cleaned from emoji ------------------------------- #

    def stemmer(self):

        tokens = word_tokenize(self.initial_text_without_emoji, language="russian")

        tokens_lower = [i.lower() for i in tokens]

        tokens_without_punctuation = [i for i in tokens_lower if i not in string.punctuation]
        self.tokenized_text = [i for i in tokens_without_punctuation if i not in russian_stop_words]
        snowball = SnowballStemmer("russian")
        self.stemmed_text = [snowball.stem(i) for i in self.tokenized_text]

    # ---------------------------- Stemmed text cleaning and preparation ------------------------------- #

    def text_cleaner(self):
        
        for item in self.stemmed_text:
            if len(item) <= 3 or item in russian_stop_words:
                
                index = self.stemmed_text.index(item)

                try:
                    del self.stemmed_text[index]
                    del self.tokenized_text[index]
                except IndexError:
                    pass
                
        for item in self.stemmed_text:
            if item in russian_stop_words:
                self.stemmed_text.remove(item)
                
    # ---------------------------- Created N-gramms ------------------------------- #

    def n_gramming(self, n):

        ngrams_stemmed = ngrams(self.stemmed_text, n)
        for grams in ngrams_stemmed:
            self.list_gramms_stemmed.append(grams)

        ngrams_tokens = ngrams(self.tokenized_text, n)
        for grams in ngrams_tokens:
            self.list_gramms_tokens.append(grams)

    # ---------------------------- Find matches ------------------------------- #

    def n_gramms_finder(self, n):

        for stemmed_tuple in self.list_gramms_stemmed:

            holder = []
            counter = 0

            for index, tup in enumerate(self.list_gramms_tokens):
                flag = False

                for item_stemmed, item_list in zip(stemmed_tuple, tup):
                    if item_stemmed in item_list:
                        flag = True
                    else:
                        flag = False
                        break

                if flag:
                    counter += 1
                    holder.append(tup)

                if counter > 1:
                    self.dict_gramms[stemmed_tuple] = (n, counter, holder)

        sorted_values = sorted(self.dict_gramms.values(), reverse=True)

        for i in sorted_values:
            for k in self.dict_gramms.keys():
                if self.dict_gramms[k] == i:
                    self.sorted_dict_gramms[k] = self.dict_gramms[k]
                    break

    # ---------------------------- Exclude matched n - gramms from n gramm stemmed list ----------------------------- #

    def n_gramms_stemmed_excluder(self, n):

        for key in self.sorted_dict_gramms.keys():
            for i in range(self.sorted_dict_gramms[key][1]):

                try:
                    index = self.list_gramms_stemmed.index(key)
                    self.list_gramms_stemmed.remove(key)
                except ValueError:
                    pass
                else:
                    if index != 0 and index != len(self.list_gramms_stemmed):
                        self.list_gramms_stemmed[index - 1] = list(self.list_gramms_stemmed[index - 1])
                        self.list_gramms_stemmed[index] = list(self.list_gramms_stemmed[index])
                        self.list_gramms_stemmed[index - 1] = self.list_gramms_stemmed[index - 1][0]
                        self.list_gramms_stemmed[index] = self.list_gramms_stemmed[index][n - 1]
                        self.list_unigramm_stemmed.append(self.list_gramms_stemmed[index - 1])
                        self.list_unigramm_stemmed.append(self.list_gramms_stemmed[index])
                        del self.list_gramms_stemmed[index]
                        del self.list_gramms_stemmed[index - 1]

                    if index == 0:
                        self.list_gramms_stemmed[index] = list(self.list_gramms_stemmed[index])
                        self.list_gramms_stemmed[index] = self.list_gramms_stemmed[index][n - 1]
                        self.list_unigramm_stemmed.append(self.list_gramms_stemmed[index])
                        del self.list_gramms_stemmed[index]

                    if index == len(self.list_gramms_stemmed):
                        self.list_gramms_stemmed[-1] = list(self.list_gramms_stemmed[-1])
                        self.list_gramms_stemmed[-1] = self.list_gramms_stemmed[-1][0]
                        self.list_unigramm_stemmed.append(self.list_gramms_stemmed[-1])
                        del self.list_gramms_stemmed[-1]

    # --------------------------- Exclude matched n gramms from n gramm tokenized list ---------------------------- #

    def n_gramms_tokens_excluder(self, n):

        for key in self.sorted_dict_gramms.keys():
            for item in self.sorted_dict_gramms[key][2]:

                try:
                    index = self.list_gramms_tokens.index(item)
                    self.list_gramms_tokens.remove(item)
                except ValueError:
                    pass
                else:
                    if index != 0 and index != len(self.list_gramms_tokens):
                        self.list_gramms_tokens[index - 1] = list(self.list_gramms_tokens[index - 1])
                        self.list_gramms_tokens[index] = list(self.list_gramms_tokens[index])
                        self.list_gramms_tokens[index - 1] = self.list_gramms_tokens[index - 1][0]
                        self.list_gramms_tokens[index] = self.list_gramms_tokens[index][n - 1]
                        self.list_unigramm_tokens.append(self.list_gramms_tokens[index - 1])
                        self.list_unigramm_tokens.append(self.list_gramms_tokens[index])
                        del self.list_gramms_tokens[index]
                        del self.list_gramms_tokens[index - 1]

                    if index == 0:
                        self.list_gramms_tokens[index] = list(self.list_gramms_tokens[index])
                        self.list_gramms_tokens[index] = self.list_gramms_tokens[index][n - 1]
                        self.list_unigramm_tokens.append(self.list_gramms_tokens[index])
                        del self.list_gramms_tokens[index]

                    if index == len(self.list_gramms_tokens):
                        self.list_gramms_tokens[-1] = list(self.list_gramms_tokens[-1])
                        self.list_gramms_tokens[-1] = self.list_gramms_tokens[-1][0]
                        self.list_unigramm_tokens.append(self.list_gramms_tokens[-1])
                        del self.list_gramms_tokens[-1]

    # --------------------------- Removing duplicate n gramms ---------------------------- #

    def cleaner(self, n):

        self.list_gramms_stemmed = self.list_gramms_stemmed[0::n]
        self.list_gramms_tokens = self.list_gramms_tokens[0::n]

    def tuple_collector(self, text):

        return " ".join(text)

    # --------------------------- Modifier from n gramms to text  ---------------------------- #

    def text_collector(self):

        self.stemmed_text = " ".join(list(map(self.tuple_collector, self.list_gramms_stemmed)))
        self.tokenized_text = " ".join(list(map(self.tuple_collector, self.list_gramms_tokens)))

        self.stemmed_text = self.stemmed_text.split()
        self.tokenized_text = self.tokenized_text.split()

    # --------------------------- Store and clean final dictionary  ---------------------------- #

    def store_and_clean(self):

        self.final_dict.update(self.sorted_dict_gramms)

        self.dict_gramms = {}
        self.sorted_dict_gramms = {}

        self.list_gramms_stemmed = []
        self.list_gramms_tokens = []

    # --------------------------- Colect unigramms  ---------------------------- #

    def collect_unigrams(self):

        self.stemmed_text += self.list_unigramm_stemmed
        self.tokenized_text += self.list_unigramm_tokens
        
    # ---------------------------- Unigramms cleaner --------------------------- #
    
    def unigramm_cleaner(self):
        for item in self.list_unigramm_stemmed:

            if len(item) <= 3 or item in russian_stop_words:
                
                index = self.list_unigramm_stemmed.index(item)

                try:
                    del self.list_unigramm_stemmed[index]
                    del self.list_unigramm_tokens[index]
                except IndexError:
                    pass                

In [13]:
def stop_words_collecter():
    
    with open("stop_words.txt", mode="r", encoding='utf-8') as stop_words_file:
        russian_stop_words = stop_words_file.readlines()
    for i in range(len(russian_stop_words) - 1):
        russian_stop_words[i] = russian_stop_words[i][:-1]

    return russian_stop_words

def user_stop_words_collecter():
    
    with open("user_stop_words.txt", mode="r", encoding='utf-8') as user_stop_words_file:
        user_stop_words = user_stop_words_file.readlines()
    for i in range(len(user_stop_words) - 1):
        user_stop_words[i] = user_stop_words[i][:-1]
        
    return user_stop_words

In [30]:
def process(n, r):
    
    my_text = docx2txt.process("text.docx")
    
    is_on = True

    first_n = n
    text_processor = Text_n(my_text)
    
    while is_on:

        if n == first_n:
            text_processor.emoji_cleaner()
            text_processor.stemmer()
            text_processor.n_gramming(n)
            text_processor.n_gramms_finder(n)
            text_processor.n_gramms_stemmed_excluder(n)
            text_processor.n_gramms_tokens_excluder(n)
            text_processor.cleaner(n)
            text_processor.text_collector()
            text_processor.store_and_clean()
            n -= 1

        if n == 1:
            text_processor.text_cleaner()
            text_processor.unigramm_cleaner()
            text_processor.collect_unigrams()
            text_processor.n_gramming(n)
            text_processor.n_gramms_finder(n)
            text_processor.store_and_clean()
            n -= 1

        else:
                  
            text_processor.n_gramming(n)
            text_processor.n_gramms_finder(n)
            text_processor.n_gramms_stemmed_excluder(n)
            text_processor.n_gramms_tokens_excluder(n)
            text_processor.cleaner(n)
            text_processor.text_collector()
            text_processor.store_and_clean()
            n -= 1

        if n == 0:
            is_on = False

    column_1 = []
    column_2 = []
    column_3 = []
    column_4 = []
    
    for item in text_processor.final_dict:
        joined = " ".join(item)
        column_1.append(joined)
        column_2.append(text_processor.final_dict[item][0])
        column_3.append(text_processor.final_dict[item][1])
        
        all_expressions = []
        
        for i in text_processor.final_dict[item][2]:
            joined_tuple = " ".join(i)
            all_expressions.append(joined_tuple)
        column_4.append(", ".join(all_expressions))

    data = {"N-gram": column_1, "N": column_2, "Number": column_3, "Expressions": column_4}

    df = pd.DataFrame(data)
    df.sort_values(['N-gram', 'Number'], ascending=[True, False])
    df = df[df['Number'] > r].reset_index(drop=True)
    file_name = "N_grams.xlsx"
    df.to_excel(file_name)

In [31]:
# ---------------------------- Собираем стоп-слова --------------------------- #
russian_stop_words = stop_words_collecter()
user_words = user_stop_words_collecter()
russian_stop_words += user_words

# ---------------------------- Выполняем задачу --------------------------- #
process(n = 3, r = 5)